In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1716494797375_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Create or retrieve a Spark session
spark = SparkSession.builder \
    .appName("Online Education Analysis") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.pyspark.python", "python3") \
    .config("spark.pyspark.virtualenv.enabled", "true") \
    .config("spark.pyspark.virtualenv.type", "native") \
    .config("spark.pyspark.virtualenv.bin.path", "/usr/bin/virtualenv") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def load_data(file_name):
    """ Load data from CSV file on S3 using Spark """
    s3_path = f"s3a://kz-final-project/{file_name}"
    return spark.read.csv(s3_path, header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def preprocess_data(df):
    """ Preprocess data by converting timestamps and filtering records """
    df = df.withColumn('timestamp_TW', to_timestamp(col('timestamp_TW')))
    return df.filter(col('is_correct') == True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def count_user_attempts(df):
    """ Count the number of attempts per user """
    return df.groupBy('uuid').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Load data
info_content_df = load_data('Info_Content.csv')
info_user_data_df = load_data('Info_UserData.csv')
log_problem_df = load_data('Log_Problem.csv')

# Process data
processed_df = preprocess_data(log_problem_df)

# Analyze data
user_attempts = count_user_attempts(processed_df)
user_attempts.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|                uuid|count|
+--------------------+-----+
|XzjV7d232DDbc3fwZ...| 1381|
|mSxpLrWH1dwKumKuP...|   82|
|e+ritEYXMn9OE97kj...|  610|
|nVR2dN2v+4pNErS9k...|  144|
|zr+bafKBCxuS4QqLC...|  415|
|0r0JQhzLmXSnXQy/O...|  397|
|i+5HEweslnkJx/DFW...|  962|
|cXpVxuFVbcITI4mcw...|  751|
|b8z8nmT8NPGkv27oP...|  136|
|SVurddKEOBdHB2R6W...|   29|
+--------------------+-----+
only showing top 10 rows

In [7]:
# Save processed data to S3
processed_df.write.parquet("s3://kz-final-project/processed_data/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…